# Calculator Applet

Demo that does various calculations - sha, hmac, pbkdf, field math, elliptic curve math

In [1]:
from applets.core import get_connection, AppletBase

class Calculator(AppletBase):
    def __init__(self, connection=None):
        super().__init__("B00B5111CC01", connection)

    def sha256(self, d):
        data = bytes([len(d)])+d
        r = "B0A10000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def hmac_sha256(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A20000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def sha512(self, d):
        data = bytes([len(d)])+d
        r = "B0A30000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def hmac_sha512(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A40000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)

    def add_mod_FP(self, a, b):
        data = bytes([len(a)])+a+bytes([len(b)])+b
        r = "B0A50000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def add_mod_N(self, a, b):
        data = bytes([len(a)])+a+bytes([len(b)])+b
        r = "B0A60000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)

In [2]:
connection = get_connection()
if connection is not None:
    atr=bytes(connection.getATR())
    print(atr.hex())
else:
    print("Failed to open a connection. No cardreaders?")

3bdc18ff8191fe1fc38073c821136605036351000250


In [3]:
app = Calculator(connection)
app.select()

b''

In [4]:
d = b'hello'
k = b'key'

In [5]:
# sha functions from the card
print("sha256:\t\t", app.sha256(d).hex())
print("hmac-sha256:\t", app.hmac_sha256(k, d).hex())
print("sha512:\t\t", app.sha512(d).hex())
print("hmac-sha512:\t", app.hmac_sha512(k, d).hex())

sha256:		 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
hmac-sha256:	 9307b3b915efb5171ff14d8cb55fbcc798c6c0ef1456d66ded1a6aa723a58b7b
sha512:		 9b71d224bd62f3785d96d46ad3ea3d73319bfbc2890caadae2dff72519673ca72323c3d99ba5c11d7c7acc6e14b8c5da0c4663475c2e5c3adef46f73bcdec043
hmac-sha512:	 ff06ab36757777815c008d32c8e14a705b4e7bf310351a06a23b612dc4c7433e7757d20525a5593b71020ea2ee162d2311b247e9855862b270122419652c0c92


In [ ]:
import hashlib, hmac
# same using hashlib
print("sha256:\t\t",hashlib.sha256(d).digest().hex())
print("hmac-sha256:\t",hmac.new(k, d, digestmod=hashlib.sha256).digest().hex())
print("sha512:\t\t",hashlib.sha512(d).digest().hex())
print("hmac-sha512:\t",hmac.new(k, d, digestmod=hashlib.sha512).digest().hex())

In [ ]:
FP = 2**256 - 2**32 - 977
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

In [ ]:
def to_bytes(n:int):
    return n.to_bytes(32, 'big')

In [ ]:
# random check
import random
for i in range(10):
    a = random.randint(1,N)
    b = random.randint(1,N)
    r = (a+b) % FP
    rr = r.to_bytes(32, 'big').hex()
    rc = app.add_mod_FP(to_bytes(a),to_bytes(b)).hex()
    print(i)
    if (rr!=rc):
        print(to_bytes(a).hex())
        print(to_bytes(b).hex())
        print(rr)
        print(rc)
        raise RuntimeError("Meh")

In [ ]:
# edge cases - slightly less than mod, slightly more, equal and overflow
a = FP-20
for i, b in enumerate([10,23,20,10500, 1020203124]):
    r = (a+b) % FP
    rr = r.to_bytes(32, 'big').hex()
    rc = app.add_mod_FP(to_bytes(a),to_bytes(b)).hex()
    print(i)
    if (rr!=rc):
        print(to_bytes(a).hex())
        print(to_bytes(b).hex())
        print(rr)
        print(rc)
        raise RuntimeError("Meh")

In [ ]:
from embit import bip32, ec
import os
seed = os.urandom(64)
root = bip32.HDKey.from_seed(seed)
# without prefix, depth, root fingerprint and index
root.serialize()[13:].hex()

In [ ]:
res = root.child(10, hardened=False).serialize()[13:].hex()
res

In [ ]:
data = bytes([65])+root.serialize()[13:]+bytes([4,0x00,0,0,10])
r = "B0A90000"+(bytes([len(data)])+data).hex().upper()
res2 = app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh")
res2

In [ ]:
# point addition
p = root.to_public().key
p.compressed = False
print(p.sec().hex())
pp = ec.PublicKey.parse(p.serialize())
ec.secp256k1.ec_pubkey_tweak_add(pp._point, b'4'*32)
res = pp.sec().hex()
print(res)

In [ ]:
data = bytes([32])+b'4'*32+bytes([65])+p.sec()
r = "B0A70000"+(bytes([len(data)])+data).hex().upper()
res2 = app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh")
res2

In [ ]:
p3 = ec.PublicKey.parse(ec.secp256k1.ec_pubkey_serialize(ec.secp256k1.ec_pubkey_combine(p._point,pp._point)))
p3.compressed = False
res = p3.sec().hex()
res

In [ ]:
data = bytes([65])+pp.sec()+bytes([65])+p.sec()
r = "B0A80000"+(bytes([len(data)])+data).hex().upper()
res2 = app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh")
res2

In [ ]:
xpub = root.to_public()
xpub.serialize()[13:].hex()

In [ ]:
res=xpub.child(10, hardened=False).serialize()[13:].hex()
res

In [ ]:
data = bytes([65])+xpub.serialize()[13:]+bytes([4,0x00,0,0,10])
r = "B0AA0000"+(bytes([len(data)])+data).hex().upper()
res2=app.request(r).hex()
if res2!=res:
    raise RuntimeError("Meh",res2)
res2

In [ ]:
import hashlib
print(hashlib.pbkdf2_hmac('sha512', b'pwd', b'salt', 2048, 64).hex())
print(hashlib.pbkdf2_hmac('sha512', b'pwd', b'salt', 100, 64).hex())

In [ ]:
import time
time.time()

In [ ]:
t0 = time.time()
# 2048 iterations -> 0x8000
iterations = 100
data = bytes([2])+iterations.to_bytes(2,'big')+bytes([3])+b'pwd'+bytes([4])+b'salt'
r = "B0A00000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())
dt = time.time()-t0
print(f"{dt} seconds for {iterations} iterations.\nFor 2048 iterations it will be {dt*2048/iterations} seconds")

In [ ]:
import os
a = os.urandom(32)
pow(int.from_bytes(a,'big'),2,FP).to_bytes(32,'big').hex()

In [ ]:
data = bytes([32])+a
r = "B0AB0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())


In [ ]:
pow(int.from_bytes(a,'big'),3,FP).to_bytes(32,'big').hex()

In [ ]:
data = bytes([32])+a
r = "B0AC0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())

In [ ]:
pow(int.from_bytes(a,'big'),FP-2,FP).to_bytes(32,'big').hex()

In [ ]:
data = bytes([32])+a
r = "B0AD0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())

In [ ]:
pow(int.from_bytes(a,'big'),2,N).to_bytes(32,'big').hex()

In [ ]:
data = bytes([32])+a
r = "B0AE0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())


In [ ]:
pow(int.from_bytes(a,'big'),3,N).to_bytes(32,'big').hex()

In [ ]:
data = bytes([32])+a
r = "B0AF0000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())


In [ ]:
pow(int.from_bytes(a,'big'),N-2,N).to_bytes(32,'big').hex()

In [ ]:
data = bytes([32])+a
r = "B0B00000"+(bytes([len(data)])+data).hex().upper()
# print(r)
print(app.request(r).hex())


In [ ]:
pub = ec.PrivateKey(os.urandom(32)).get_public_key()
sec = pub.sec()
pub.compressed=False
sec_unc = pub.sec()
sec_unc.hex()

In [ ]:
data = bytes([65])+sec_unc
r = "B0B10000"+(bytes([len(data)])+data).hex().upper()
# print(r)
res = app.request(r)
print(res.hex())
res == sec

In [ ]:
data = bytes([33])+sec
r = "B0B20000"+(bytes([len(data)])+data).hex().upper()
# print(r)
res = app.request(r)
print(res.hex())
res == sec_unc

In [6]:
# signature check
from embit.ec import Signature, secp256k1

In [17]:
badsig = Signature(b'\xFA'*32+b'\x1F'*29+b'\xFF\xFF\xFF')
print(badsig.serialize().hex())
sig = Signature(secp256k1.ecdsa_signature_normalize(badsig._sig))
print(sig.serialize().hex())

3046022100fafafafafafafafafafafafafafafafafafafafafafafafafafafafafafafafa022100ffffff1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f
3043022100fafafafafafafafafafafafafafafafafafafafafafafafafafafafafafafafa021e00e0e0e0e0e0e0e0e0e0e0e0e0df9b8fbdc79029811ca0b33f6db1172222


In [18]:
der = badsig.serialize()
data = bytes([len(der)])+der
r = "B0B30000"+(bytes([len(data)])+data).hex().upper()
print(der.hex())
res = app.request(r)[6:]
print(sig.serialize().hex())
print(res.hex())
res==sig.serialize()

3046022100fafafafafafafafafafafafafafafafafafafafafafafafafafafafafafafafa022100ffffff1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f1f
3043022100fafafafafafafafafafafafafafafafafafafafafafafafafafafafafafafafa021e00e0e0e0e0e0e0e0e0e0e0e0e0df9b8fbdc79029811ca0b33f6db1172222
3043022100fafafafafafafafafafafafafafafafafafafafafafafafafafafafafafafafa021e00e0e0e0e0e0e0e0e0e0e0e0e0df9b8fbdc79029811ca0b33f6db1172222


True

In [19]:
connection.disconnect()